#  Programming Languages (6) --- How compilers translate your program


Enter your name and student ID.

 * Name:
 * Student ID:


<a name="intro"> </a>
# 1. Objective
* you look at assembly code generated by the compiler to get a good sense of how source programs are compiled into machine (assembly) code

* coding exercise should be done in this notebook
* answers to questions should be done in the OneDrive folder (each team should have `pl06_how_it_gets_compiled` folder)

# 2. Roadmap
# 3. Prepare AI Tutor
* execute the following cell to set up your tutor

In [ ]:
from heytutor import *
config(default_lang=" ... ")  # choose one of Go/Julia/OCaml/Rust

* you may want to ask a few basics

In [ ]:
hey("what is adrp instruction of ARM64?")
# hey("what is ARM64 for integer division")

# 4. Let the compiler generate assembly code
* when you compile a program or build a project, a compiler generally compiles it all the way into an executable
* that is, machine (or assembly) code is generated during the process, but is discarded after the compilation
* you first have to know how to let your compiler emit and leave assembly code 
* how to do that exactly is language- or compiler-dependent

## 4-1. Go
### 4-1-1. A simple example
* when executed by SHIFT + ENTER, the following cell simply writes the content (excluding the first `%%writefile go/add123.go`) into the specified file `go/add123.go`
* note that it is a feature of Python kernel and you don't have to (and should not) change it to `Go` kernel


In [ ]:
%%writefile go/add123.go

package pl06
func Add123(x int64) int64 {
    return x + 123;
}

### 4-1-2. How to emit assembly code
* ordinary go compiler does not support emitting native assembly
* it instead emits something called "plan 9 assembly" ([details](https://go.dev/doc/asm)), which is a machine-independent, therefore portable, assembly
* to obtain the native assembly, you first `go build` a file to obtain an _object file_ --- the real binary machine code containing instructions --- and reverse it back to assembly (textual representation of instructions) using `go tool objdump`
* `go tool objdump` prints the plan 9 assembly; giving `-gnu` option prints the native assembly as well, which is our interest
* <font color="red">Note:</font> to make the output easy to read, shrink the text size so that lines do not wrap


In [ ]:
go build -o go/add123.o go/add123.go
go tool objdump -gnu go/add123.o

* **NOTE:**
  * if the command does not output anything, do it again (likely a bash kernel bug)

<!---
* portability is good, but for the learning objective of this exercise, it obscures how your program executes on the real machine
* therefore, in this exercise, we instead use an alternative compiler, `gccgo`, which generates a native assembly for x86
  * `-S` to emit assembly code
  * `-O` to optimize
  * `-g0` not to generate debugging information (which is unncessary now)

<!- -- code w kernel=bash -- ->
gccgo -O -g0 -S go/add123.go -o go/add123.s
cat -n go/add123.s
<!- -- end code -- ->

--->

* try to locate the instruction that does `x + 123`
* try to locate the start of the function, which should be labeled with a symbol similar to the function name `Add123` (it may not be exactly the same)

* **NOTE:**
  * you need to say `package xxx`, with _xxx_ anything other than `main`; if you say `package main`, the compiler complains about the lack of `main` function
  * you need to <font color="red">capitalize the function name</font> whose assembly code you want to see, as otherwise the compiler may eliminate it as a dead code
  * in Go, functions starting with an uppercase letter are visible (can be called) from another package (see https://go.dev/doc/tutorial/create-module), so the compiler should leave the assembly code even if it finds the function not called anywhere in the file it is compiling

<!---

* **NOTE 2:** if you want to use `gccgo` along with `go build`, you can do
```
go build -compiler gccgo 
```
instead of just 
```
go build
```
--->

## 4-2. Julia
### 4-2-1. A simple example
* when executed by SHIFT + ENTER, the following cell simply writes the content (excluding the first `%%writefile jl/add123.jl`) into the specified file `jl/add123.jl`
* note that it is a feature of Python kernel and you don't have to (and should not) change it to `Julia` kernel


In [ ]:
%%writefile jl/add123.jl

function add123(n :: Int64)
    n + 123
end

using InteractiveUtils
code_native(add123)

### 4-2-2. How to emit assembly code
* Julia has <font color="blue">code_native</font> function that prints assembly code of a function given particular concrete types for its input parameters
* running this program defines `add123` and then calls `code_native` function, which shows assembly code for a specified function


In [ ]:
julia jl/add123.jl

* you can alternatively write only the definition of a function in the `.jl` file and call `code_native` in the command line


In [ ]:
%%writefile jl/add123.jl

function add123(n :: Int64)
    n + 123
end

In [ ]:
julia --load jl/add123.jl --eval 'using InteractiveUtils; code_native(add123)'

* try to locate the instruction that does `x + 123`
* try to locate the start of the function, which should be labeled with a symbol similar to the function name `add123` (it may not be exactly the same)

* note that the function `add123` declares the type of the input parameter (`n`) as `Int64`; you could give a different type or omit its type altogether and see how the generated coce change
* you can alternatively specify input type as the second parameter of `code_native`, like `code_native(add123, (Int64,))` or `code_native(add123, (Float64,))` in which case you perhaps want to omit the type in the definition

## 4-3. OCaml
### 4-3-1. A simple example
* when executed by SHIFT + ENTER, the following cell simply writes the content (excluding the first `%%writefile ml/add123.ml`) into the specified file `ml/add123.ml`
* note that it is a feature of Python kernel and you don't have to (and should not) change it to `OCaml` kernel


In [ ]:
%%writefile ml/add123.ml

let add123 n = n + 123
;;

### 4-3-2. How to emit assembly code
* `ocamlopt` is a "native compiler" that translates OCaml source into machine code (`ocamlc` compiles OCaml to bytecode)
  * `-S` to emit assembly code


In [ ]:
ocamlopt -S ml/add123.ml
cat -n ml/add123.s

* try to locate the instruction that does `x + 123`
* try to locate the start of the function, which should be labeled with a symbol similar to the function name `add123` (it may not be exactly the same)

## 4-4. Rust
### 4-4-1. A simple example
* when executed by SHIFT + ENTER, the following cell simply writes the content (excluding the first `%%writefile rs/add123.rs`) into the specified file `rs/add123.rs`
* note that it is a feature of Python kernel and you don't have to (and should not) change it to `Rust` kernel


In [ ]:
%%writefile rs/add123.rs

#[no_mangle]
pub fn add123(x : i64) -> i64 {
    x + 123
}

### 4-4-2. How to emit assembly code
* `rustc` is a compiler for Rust, which is internally called by `cargo`
  * `-O` to optimize
  * `--emit asm` to generate assembly code
  * `--crate-type lib` to say this is for a library, not an executable


In [ ]:
rustc -O --emit asm --crate-type lib rs/add123.rs -o rs/add123.s
cat -n rs/add123.s

* try to locate the instruction that does `x + 123`
* try to locate the start of the function, which should be labeled with a symbol similar to the function name `add123` (it may not be exactly the same)

* **NOTE:**
  * `--crate-type lib` says you are building a library which will be called from another program, not an executable; without it, the compiler complains about the lack of `main` function
  * `#[no_mangle]` guarantees the label name corresponding to `add123` is exactly `add123` and has a side-effect of preventing the compiler from eliminating the function as dead code (I try to find a way to guarantee `add123` is not eliminatd without `#[no_mangle]`, but couldn't.  let me know if you know a better way)

# <font color="green"> Problem 1 :  Calling convention (where are args and return values?)</font>
* define a function `many_args` (or `Many_args` in Go) that takes many (> 10) integer parameters and returns the sum of them (similar to the following C function) in your language, compile it into assembly and examine it


In [ ]:
%%writefile c/many_args.c

long many_args(long a00, long a01, long a02, long a03, long a04, long a05,
               long a06, long a07, long a08, long a09, long a10, long a11) {
  return a00 + a01 + a02 + a03 + a04 + a05 + a06 + a07 + a08 + a09 + a10 + a11;
}

In [ ]:
gcc -O3 -S c/many_args.c -o c/many_args.s
cat -n c/many_args.s

* write the code below
* <font color="red">replace xx with go, jl, ml, or rs and write the function</font>


In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile xx/many_args.xx


* and see the assembly code by executing one of the following

* Go


In [ ]:
go build -o go/many_args.o go/many_args.go
go tool objdump -gnu go/many_args.o

<!---
<!- -- code w kernel=bash -- ->
gccgo -O -g0 -S go/many_args.go -o go/many_args.s
cat -n go/many_args.s
<!- -- end code -- ->
--->

* Julia


In [ ]:
julia --load jl/many_args.jl --eval 'using InteractiveUtils; code_native(many_args)'

* OCaml


In [ ]:
ocamlopt -S ml/many_args.ml
cat -n ml/many_args.s

* Rust


In [ ]:
rustc -O --emit asm --crate-type lib rs/many_args.rs -o rs/many_args.s
cat -n rs/many_args.s

* __<font color="blue">Questions:</font>__
  1. how are parameters passed in your language? which registers? memory?
  1. how is the return value passed in your language? which registers? memory?
  1. discuss differences between languages in the team and write notable difference(s) you found

<font color="red">write your answers in the Excel sheet in OneDrive folder given to each team</font> (OneDrive : `pl06_how_it_gets_compiled/questions.xlsx`)

# <font color="green"> Problem 2 :  How floating point numbers are represented</font>
* define a function `add_floats` (or `Add_floats` in Go) that takes two floating point numbers and returns the sum of the two (similar to the following C function) in your language, compile it into assembly and examine it


In [ ]:
%%writefile c/add_floats.c

double add_floats(double x, double y) {
  return x + y;
}

In [ ]:
gcc -O3 -S c/add_floats.c -o c/add_floats.s
cat -n c/add_floats.s

* write the code below
* <font color="red">replace xx with go, jl, ml, or rs and write the function</font>


In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile xx/add_floats.xx


* and see the assembly code by executing one of the following

* Go


In [ ]:
go build -o go/add_floats.o go/add_floats.go
go tool objdump -gnu go/add_floats.o

<!---
<!- -- code w kernel=bash -- ->
gccgo -O -g0 -S go/add_floats.go -o go/add_floats.s
cat -n go/add_floats.s
<!- -- end code -- ->
--->

* Julia


In [ ]:
julia --load jl/add_floats.jl --eval 'using InteractiveUtils; code_native(add_floats)'

* OCaml


In [ ]:
ocamlopt -S ml/add_floats.ml
cat -n ml/add_floats.s

* Rust


In [ ]:
rustc -O --emit asm --crate-type lib rs/add_floats.rs -o rs/add_floats.s
cat -n rs/add_floats.s

* __<font color="blue">Questions:</font>__
  1. how are the two parameters passed in your language? which registers? memory?
  1. how does your language represent a floating point number $x$?
  1. discuss difference between languages in the team and write notable difference(s) you found


# <font color="green"> Problem 3 :  How structs are represented</font>
* define a struct `point` (or `Point`) that has two fields (`x` and `y`; both are 64 bit floating point numbers) and function `get_point_y` (or `Get_point_y`) that takes a point or a pointer to it and returns its `y` field


In [ ]:
%%writefile c/get_struct_elem.c

typedef struct {
  double x;
  double y;
} point;
double get_point_y(point p) {
  return p.y;
}
double get_pointp_y(point * p) {
  return p->y;
}

In [ ]:
gcc -O3 -S c/get_struct_elem.c -o c/get_struct_elem.s
cat -n c/get_struct_elem.s

* write the code below
* <font color="red">replace xx with go, jl, ml, or rs and write the function</font>


In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile xx/get_struct_elem.xx


* and see the assembly code by executing one of the following

* Go


In [ ]:
go build -o go/get_struct_elem.o go/get_struct_elem.go
go tool objdump -gnu go/get_struct_elem.o

<!---
<!- -- code w kernel=bash -- ->
gccgo -O -g0 -S go/get_struct_elem.go -o go/get_struct_elem.s
cat -n go/get_struct_elem.s
<!- -- end code -- ->
--->

* Julia


In [ ]:
julia --load jl/get_struct_elem.jl --eval 'using InteractiveUtils; code_native(get_point_y)'

* OCaml


In [ ]:
ocamlopt -S ml/get_struct_elem.ml
cat -n ml/get_struct_elem.s

* Rust


In [ ]:
rustc -O --emit asm --crate-type lib rs/get_struct_elem.rs -o rs/get_struct_elem.s
cat -n rs/get_struct_elem.s

* __<font color="blue">Questions:</font>__
  1. how does your language represent `point` (or `Point`)?
  1. how does Go represent `*Point`?  is it different from `Point`?
  1. how does Rust represent `&Point` and `Box::<Point>`?  are they different from `Point`?  is there a difference between them?
  1. discuss differences between languages in the team and write notable difference(s) you found

# <font color="green"> Problem 4 :  How arrays are represented</font>
* define a function `get_float_array_elem` (or `get_float_array_elem` in Go) that takes an array of floating point numbers, `a`, and an integer, `i`, and returns `i`-th element of `a` (similar to to the following C function) in your language, compile it into assembly and examine it
* for an array, use
  * `[]float64` (slice) for Go
  * `Vector{Float64}` for Julia
  * `float array` (slice) for OCaml
  * `Vec<f64>` for Rust


In [ ]:
%%writefile c/get_float_array_elem.c

double get_float_array_elem_const(double a[10]) {
  return a[2];
}
double get_float_array_elem_i(double a[10], long i) {
  return a[i];
}

In [ ]:
gcc -O3 -S c/get_float_array_elem.c -o c/get_float_array_elem.s
cat -n c/get_float_array_elem.s

* write the code below
* <font color="red">replace xx with go, jl, ml, or rs and write the function</font>


In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile xx/get_float_array_elem.xx


* and see the assembly code by executing one of the following

* Go


In [ ]:
go build -o go/get_float_array_elem.o go/get_float_array_elem.go
go tool objdump -gnu go/get_float_array_elem.o

<!---
<!- -- code w kernel=bash -- ->
gccgo -O -g0 -S go/get_float_array_elem.go -o go/get_float_array_elem.s
cat -n go/get_float_array_elem.s
<!- -- end code -- ->
--->

* Julia


In [ ]:
julia --load jl/get_float_array_elem.jl --eval 'using InteractiveUtils; code_native(get_float_array_elem_const)'

In [ ]:
julia --load jl/get_float_array_elem.jl --eval 'using InteractiveUtils; code_native(get_float_array_elem_i)'

* OCaml


In [ ]:
ocamlopt -S ml/get_float_array_elem.ml
cat -n ml/get_float_array_elem.s

* Rust


In [ ]:
rustc -O --emit asm --crate-type lib rs/get_float_array_elem.rs -o rs/get_float_array_elem.s
cat -n rs/get_float_array_elem.s

* __<font color="blue">Questions:</font>__
  1. locate the instruction that gets the element from memory
  1. how does your language represent the respective array-like data structure?
  1. discuss differences between languages in the team and write notable difference(s) you found

# <font color="green"> Problem 5 :  If statement/expressions</font>
* define a function `collatz` (or `Collatz` in Go) that takes an integer $n$ and returns $n/2$ if $n$ is an even number and $3 n + 1$ otherwise (similar to the following C function)


In [ ]:
%%writefile c/collatz.c

long collatz(long n) {
  if (n % 2 == 0) {
    return n / 2;
  } else {
    return 3 * n + 1;
  }
}

In [ ]:
gcc -O3 -S c/collatz.c -o c/collatz.s
cat -n c/collatz.s

* write the code below
* <font color="red">replace xx with go, jl, ml, or rs and write the function</font>


In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile xx/collatz.xx


* and see the assembly code by executing one of the following

* Go


In [ ]:
go build -o go/collatz.o go/collatz.go
go tool objdump -gnu go/collatz.o

<!---
<!- -- code w kernel=bash -- ->
gccgo -O -g0 -S go/collatz.go -o go/collatz.s
cat -n go/collatz.s
<!- -- end code -- ->
--->

* Julia


In [ ]:
julia --load jl/collatz.jl --eval 'using InteractiveUtils; code_native(collatz)'

* OCaml


In [ ]:
ocamlopt -S ml/collatz.ml
cat -n ml/collatz.s

* Rust


In [ ]:
rustc -O --emit asm --crate-type lib rs/collatz.rs -o rs/collatz.s
cat -n rs/collatz.s

* __<font color="blue">Questions:</font>__
  1. locate the conditional jump instruction that jumps depending on the result of the comparison
  1. discuss differences between languages in the team and write notable difference(s) you found

# <font color="green"> Problem 6 :  loops</font>
* define a function `sum_array_loop` (or `sum_array_loop` in Go) that takes an array of floating point numbers and returns the sum of all its elements (similar to to the following C function) in your language, compile it into assembly and examine it
* for OCaml, use a [for loop](https://ocaml.org/docs/loops-recursion)  and mutable variables (`ref`)
* use
  * `[]float64` (slice) in Go
  * `Vector{Float64}` in Julia
  * `float array` in OCaml
  * `Vec::<f64>` in Rust
for an array


In [ ]:
%%writefile c/sum_array_loop.c

double sum_array_loop(double * a, long n) {
  double s = 0.0;
  for (int i = 0; i < n; i++) {
    s += a[i];
  }
  return s;
}

In [ ]:
gcc -O3 -S c/sum_array_loop.c -o c/sum_array_loop.s
cat -n c/sum_array_loop.s

* write the code below
* <font color="red">replace xx with go, jl, ml, or rs and write the function</font>


In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile xx/sum_array_loop.xx


* and see the assembly code by executing one of the following

* Go


In [ ]:
go build -o go/sum_array_loop.o go/sum_array_loop.go
go tool objdump -gnu go/sum_array_loop.o

<!---
<!- -- code w kernel=bash -- ->
gccgo -O -g0 -S go/sum_array_loop.go -o go/sum_array_loop.s
cat -n go/sum_array_loop.s
<!- -- end code -- ->
--->

* Julia

In [ ]:
julia --load jl/sum_array_loop.jl --eval 'using InteractiveUtils; code_native(sum_array_loop)'

* OCaml


In [ ]:
ocamlopt -S ml/sum_array_loop.ml
cat -n ml/sum_array_loop.s

* Rust


In [ ]:
rustc -O --emit asm --crate-type lib rs/sum_array_loop.rs -o rs/sum_array_loop.s
cat -n rs/sum_array_loop.s

* __<font color="blue">Questions:</font>__
  1. given that the array has a large number of elements, identify the instruction sequence that takes most of the time
  1. in that sequence, how many instructions are executed per array element?
  1. discuss differences between languages in the team and write notable difference(s) you found

# <font color="green"> Problem 7 :  Function call</font>
* define a function `call_tanh` (or `Call_tanh` in Go) that takes a floating point number $x$ and returns $\tanh(x + 1.0) + 2.0$


In [ ]:
%%writefile c/call_tanh.c

#include <math.h>
double call_tanh(double x) {
  return tanh(x + 1.0) + x;
}

In [ ]:
gcc -O -S c/call_tanh.c -o c/call_tanh.s
cat -n c/call_tanh.s

* write the code below
* <font color="red">replace xx with go, jl, ml, or rs and write the function</font>


In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile xx/call_tanh.xx


* and see the assembly code by executing one of the following

* Go


In [ ]:
go build -o go/call_tanh.o go/call_tanh.go
go tool objdump -gnu go/call_tanh.o

<!---
<!- -- code w kernel=bash -- ->
gccgo -O -g0 -S go/call_tanh.go -o go/call_tanh.s
cat -n go/call_tanh.s
<!- -- end code -- ->
--->

* Julia


In [ ]:
julia --load jl/call_tanh.jl --eval 'using InteractiveUtils; code_native(call_tanh)'

* OCaml


In [ ]:
ocamlopt -S ml/call_tanh.ml
cat -n ml/call_tanh.s

* Rust


In [ ]:
rustc -O --emit asm --crate-type lib rs/call_tanh.rs -o rs/call_tanh.s
cat -n rs/call_tanh.s

* __<font color="blue">Questions:</font>__
  1. locate the instruction that calls `tanh` or a similar function
  1. as the function computes $\tanh(x + 1.0) + x$, the value of `x` must be saved somewhere during the function call
  1. discuss differences between languages in the team and write notable difference(s) you found

<!---
# <font color="green"> Problem 8 :  Divide and conquer</font>
* define a function `sum_array_rec` (or `sum_array_rec` in Go) that takes an array $a$ and two integers $p$ and $q$ and returns sum of $a[p:q]$
* write it in the divide and conquer recursion (similar to the following C function)

<!- -- code kernel=py -- ->
%%writefile c/sum_array_rec.c

<!- -- exec-include ./mk_version.py -D label="sum_array_rec" nb/source/pl06_how_it_gets_compiled/include/c/pl06.c -- ->
<!- -- end code -- ->

<!- -- code w kernel=bash -- ->
gcc -O -S c/sum_array_rec.c -o c/sum_array_rec.s
cat -n c/sum_array_rec.s
<!- -- end code -- ->

* write the code below
* <font color="red">replace xx with go, jl, ml, or rs and write the function</font>

<!- -- code w kernel=py points=1 -- ->
%%writefile xx/sum_array_rec.xx

<!- -- end code -- ->

* and see the assembly code by executing one of the following

* Go

<!- -- code label=ans kernel=py -- ->
%%writefile go/sum_array_rec.go

<!- -- exec-include ./mk_version.py -D label="sum_array_rec" nb/source/pl06_how_it_gets_compiled/include/go/pl06.go -- ->
<!- -- end code -- ->

<!- -- code w kernel=bash -- ->
go build -o go/sum_array_rec.o go/sum_array_rec.go
go tool objdump -gnu go/sum_array_rec.o
<!- -- end code -- ->

<!- --
<!- -- code w kernel=bash -- ->
gccgo -O -g0 -S go/sum_array_rec.go -o go/sum_array_rec.s
cat -n go/sum_array_rec.s
<!- -- end code -- ->

* Julia

<!- -- code label=ans kernel=py -- ->
%%writefile jl/sum_array_rec.jl

<!- -- exec-include ./mk_version.py -D label="sum_array_rec" nb/source/pl06_how_it_gets_compiled/include/jl/pl06.jl -- ->
<!- -- end code -- ->

<!- -- code w kernel=bash -- ->
julia --load jl/sum_array_rec.jl --eval 'using InteractiveUtils; code_native(sum_array_rec)'
<!- -- end code -- ->

* OCaml

<!- -- code label=ans kernel=py -- ->
%%writefile ml/sum_array_rec.ml

<!- -- exec-include ./mk_version.py -D label="sum_array_rec" nb/source/pl06_how_it_gets_compiled/include/ml/pl06.ml -- ->
<!- -- end code -- ->

<!- -- code w kernel=bash -- ->
ocamlopt -S ml/sum_array_rec.ml
cat -n ml/sum_array_rec.s
<!- -- end code -- ->

* Rust

<!- -- code label=ans kernel=py -- ->
%%writefile rs/sum_array_rec.rs

<!- -- exec-include ./mk_version.py -D label="sum_array_rec" nb/source/pl06_how_it_gets_compiled/include/rs/pl06.rs -- ->
<!- -- end code -- ->

<!- -- code w kernel=bash -- ->
rustc -O --emit asm --crate-type lib rs/sum_array_rec.rs -o rs/sum_array_rec.s
cat -n rs/sum_array_rec.s
<!- -- end code -- ->

* __<font color="blue">Questions:</font>__
  1. locate the instruction that recursively calls `sum_array_rec` (or `Sum_array_rec`)
  1. as the function computes `sum_array_rec(n - 1) + n - 1`, the value of `n` (or `n - 1`) must be saved somewhere during `sum_array_rec(n - 1)`; where is it saved?
  1. discuss differences between languages in the team and write notable difference(s) you found
  
--->  

# <font color="green"> Problem 9 :  Tail recursive call</font>
* define a function `sum_array_tail_rec` (or `Sum_array_tail_rec` in Go) that takes an array $a$, two integers $i$ and $n$ and a floating point number $s$, and returns $s + a[i:n]$
* write it in the _tail recursive_ manner (similar to the following C function)


In [ ]:
%%writefile c/sum_array_tail_rec.c

double sum_array_tail_rec(double * a, long i, long n, double s) {
  if (i == n) {
    return s;
  } else {
    return sum_array_tail_rec(a, i + 1, n, s + a[i]);
  }
}

In [ ]:
gcc -O2 -S c/sum_array_tail_rec.c -o c/sum_array_tail_rec.s
cat -n c/sum_array_tail_rec.s

* generally, a function call whose return value becomes the return value of the caller is called a _tail call_
* for example, the function call `g(x)` below is a tail call, but `h(x)` is not
```
def f(x):
   if ...:
      return g(x)
   else:
      return h(x) + 1
```
* a _tail call_ that is also a recursive call is called a _tail recursive call_ and a compiler generally has a chance to optimize tail recursive calls, by replacing call/return into a jump
* a loop can generally be expressed as tail recursive functions; for example,
```
while A:
  x = Ex
  y = Ey
```
can be translated into something like
```
def loop(x, y):
  if A:
     let x = Ex in
     let y = Ey in
     loop(x, y)
```
* notice that the recursive call to `loop(...)` is a tail call
* this call is hopefully compiled into a jump to somewhere near the start of the function, rather than an ordinary function calling sequence (using `call` instruction, among others)
* this is called several ways, such as "tail call optimization", "tail call elimination", or "tail recursion elimination"
* in particular, in OCaml, which has a loop but encourages more functional way of writing things, writing what would be a loop in other languages in tail recursive functions is a trick you want to master
* moreover, many language implementations have a limit on the depth of recursions due to the limitation of the stack space (details covered later), so writing a recursion in the tail-recursive manner is sometimes the only way (other than resorting for loops and mutable variables)

* write the code below
* <font color="red">replace xx with go, jl, ml, or rs and write the function</font>


In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile xx/sum_array_tail_rec.xx


* and see the assembly code by executing one of the following

* Go


In [ ]:
go build -o go/sum_array_tail_rec.o go/sum_array_tail_rec.go
go tool objdump -gnu go/sum_array_tail_rec.o

<!---
<!- -- code w kernel=bash -- ->
gccgo -O -g0 -S go/sum_array_tail_rec.go -o go/sum_array_tail_rec.s
cat -n go/sum_array_tail_rec.s
<!- -- end code -- ->
--->

* Julia


In [ ]:
julia --load jl/sum_array_tail_rec.jl --eval 'using InteractiveUtils; code_native(sum_array_tail_rec)'

* OCaml


In [ ]:
ocamlopt -S ml/sum_array_tail_rec.ml
cat -n ml/sum_array_tail_rec.s

* Rust


In [ ]:
rustc -O --emit asm --crate-type lib rs/sum_array_tail_rec.rs -o rs/sum_array_tail_rec.s
cat -n rs/sum_array_tail_rec.s

* __<font color="blue">Questions:</font>__
  1. does your language use `call` instruction or successfully eliminate it (compile it into a loop)?
  1. discuss differences between languages in the team and write notable difference(s) you found

# <font color="green"> Problem 10 :  Your investigation</font>
* pick any language construct or feature not covered above and investigate how it is implemented by writing an appropriate function(s) and translating it into assembly
* you could investigate a feature unique to your language or common across several or all languages

* write the code and a description of what you investigated below
* <font color="red">replace xx with go, jl, ml, or rs, yyyyyy with an appropriate name, and write the function</font>


BEGIN SOLUTION
END SOLUTION


In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile xx/yyyyyy.xx


* and see the assembly code by executing one of the following
